Importing Libararies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

ImportError: Traceback (most recent call last):
  File "c:\Users\Soham Kishor Walam\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

Dataset and Preprocessing

In [ ]:

# Load dataset
file_path = "traffic_data.csv"
df = pd.read_csv(file_path)

# Convert date and time to datetime
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%m/%d/%y %H:%M:%S')
df.set_index('DateTime', inplace=True)

User Input Analysis and Graph

In [ ]:
# User input for region
region_input = input("Enter region to analyze traffic trends: ")
df_region = df[df['Region'] == region_input]

if df_region.empty:
    print(f"No data available for region: {region_input}")
else:
    df_region_sorted = df_region.sort_values(by='Traffic_Index')
    
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_region_sorted, x=df_region_sorted.index.date, y='Traffic_Index', estimator=np.mean, ci=None, palette="coolwarm")
    plt.xticks(rotation=90)
    plt.title(f'Traffic Index Over Years in {region_input} (Sorted)')
    plt.show()

# High traffic occurrences
threshold = df['Traffic_Index'].median()
df_high_traffic = df[df['Traffic_Index'] > threshold].sort_values(by='Traffic_Index')

plt.figure(figsize=(12, 6))
sns.barplot(data=df_high_traffic, x=df_high_traffic.index.date, y='Traffic_Index', estimator=np.mean, ci=None, palette="Reds")
plt.xticks(rotation=90)
plt.title(f'High Traffic Index Occurrences (Threshold > {threshold}) - Sorted')
plt.show()

# Median traffic analysis
median_traffic = df['Traffic_Index'].median()
df_high_median = df[df['Traffic_Index'] > median_traffic].sort_values(by='Traffic_Index')

plt.figure(figsize=(12, 6))
sns.barplot(data=df_high_median, x=df_high_median.index.date, y='Traffic_Index', estimator=np.mean, ci=None, palette="Blues")
plt.xticks(rotation=90)
plt.title(f'Traffic Index Above Median ({median_traffic}) - Sorted')
plt.show()


Normalization

In [ ]:
# Normalize data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[['Traffic_Index']])

# Prepare sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

Model Training

In [ ]:
SEQ_LENGTH = 24
X, y = create_sequences(df_scaled, SEQ_LENGTH)
train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

Performance Metrics

In [ ]:
mae = mean_absolute_error(y_test_actual, predictions)
rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
r2 = r2_score(y_test_actual, predictions)
print(f"MAE: {mae}, RMSE: {rmse}, R2 Score: {r2}")

plt.figure(figsize=(12, 6))
plt.plot(y_test_actual, label='Actual Traffic Index')
plt.plot(predictions, label='Predicted Traffic Index', linestyle='dashed')
plt.legend()
plt.title('Traffic Flow Prediction')
plt.show()

Visualization

In [ ]:

# Accident risk analysis
df['Accident_Risk'] = df['Traffic_Index'] + df['Pollution_Index']
df_sorted = df.sort_values(by='Accident_Risk')

plt.figure(figsize=(10, 6))
colors = ['red' if row['Accident_Risk'] > 180 else 'yellow' if row['Pollution_Index'] > 120 else 'blue' if row['Max_Traffic_Time'] in ['Morning', 'Evening'] else 'green' for _, row in df_sorted.iterrows()]
plt.scatter(df_sorted.index, df_sorted['Traffic_Index'], c=colors, alpha=0.5)
plt.xlabel('Time')
plt.ylabel('Traffic Index')
plt.title('Regions with Highest Accident Risks (Sorted)')
plt.show()
